In [1]:
#r "nuget:Microsoft.ML"
#r "nuget:Microsoft.ML.TimeSeries"

Installed Packages Microsoft.ML, 4.0.2 Microsoft.ML.TimeSeries, 4.0.2

In [ ]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.TimeSeries;

In [3]:
public class AnomalyPrediction
{
    [VectorType(3)]
    public double[] Prediction { get; set; }
}

public class EventLogsCountAnomaly
{
    [LoadColumn(0)]
    public DateTime TimeStamp { get; set; } // Nullable DateOnly property
    [LoadColumn(1)]
    public float Value { get; set; } // Count property
}

In [4]:
var preprocessedData = new List<EventLogsCountAnomaly>
{
    new EventLogsCountAnomaly { TimeStamp = DateTime.Now.AddDays(-1), Value = 1200 },
    new EventLogsCountAnomaly { TimeStamp = DateTime.Now.AddDays(-2), Value = 1500 },
    new EventLogsCountAnomaly { TimeStamp = DateTime.Now.AddDays(-3), Value = 2000 },
    new EventLogsCountAnomaly { TimeStamp = DateTime.Now.AddDays(-4), Value = 1200 },
    new EventLogsCountAnomaly { TimeStamp = DateTime.Now.AddDays(-5), Value = 2000 },
    new EventLogsCountAnomaly { TimeStamp = DateTime.Now.AddDays(-6), Value = 800 },
    new EventLogsCountAnomaly { TimeStamp = DateTime.Now.AddDays(-7), Value = 300 },
    new EventLogsCountAnomaly { TimeStamp = DateTime.Now.AddDays(-8), Value = 20 },
    new EventLogsCountAnomaly { TimeStamp = DateTime.Now.AddDays(-9), Value = 4000 },
    new EventLogsCountAnomaly { TimeStamp = DateTime.Now.AddDays(-10), Value = 1500 },
    new EventLogsCountAnomaly { TimeStamp = DateTime.Now.AddDays(-11), Value = 1400 },
    new EventLogsCountAnomaly { TimeStamp = DateTime.Now.AddDays(-12), Value = 800 },
    new EventLogsCountAnomaly { TimeStamp = DateTime.Now.AddDays(-13), Value = 200 },
    new EventLogsCountAnomaly { TimeStamp = DateTime.Now.AddDays(-14), Value = 11400 },
    new EventLogsCountAnomaly { TimeStamp = DateTime.Now.AddDays(-15), Value = 1600 },
    new EventLogsCountAnomaly { TimeStamp = DateTime.Now.AddDays(-16), Value = 2200 },
    new EventLogsCountAnomaly { TimeStamp = DateTime.Now.AddDays(-17), Value = 2400 },
    new EventLogsCountAnomaly { TimeStamp = DateTime.Now.AddDays(-18), Value = 300 },
    new EventLogsCountAnomaly { TimeStamp = DateTime.Now.AddDays(-19), Value = 900 },
    new EventLogsCountAnomaly { TimeStamp = DateTime.Now.AddDays(-20), Value = 1400 }
};

In [34]:
var context = new MLContext();
var dataView = context.Data.LoadFromEnumerable(preprocessedData);

var pipeline = context.Transforms.DetectSpikeBySsa(
    outputColumnName: nameof(AnomalyPrediction.Prediction),
    inputColumnName: nameof(EventLogsCountAnomaly.Value),
    confidence: 99.0,
    pvalueHistoryLength: preprocessedData.Count / 4,
    trainingWindowSize: preprocessedData.Count,
    seasonalityWindowSize: preprocessedData.Count / 4);

var model = pipeline.Fit(dataView);
var transformedData = model.Transform(dataView);

var predictions = context.Data.CreateEnumerable<AnomalyPrediction>(transformedData, reuseRowObject: false).ToList();

Console.WriteLine("Time            tValue  Anomaly");
for (int i = 0; i < predictions.Count; i++)
{
    Console.WriteLine($"{DateOnly.FromDateTime(preprocessedData[i].TimeStamp)}\t{preprocessedData[i].Value}\t{predictions[i].Prediction[0] == 1}");
}

// Console.WriteLine("Time            tValue  Anomaly");
for (int i = 0; i < predictions.Count; i++)
{
    Console.WriteLine($"{DateOnly.FromDateTime(preprocessedData[i].TimeStamp)}\t{predictions[i].Prediction[1]}\t{predictions[i].Prediction[2]}");
}
    

Time            tValue  Anomaly
3/14/2025	1200	False
3/13/2025	1500	False
3/12/2025	2000	False
3/11/2025	1200	False
3/10/2025	2000	False
3/9/2025	800	False
3/8/2025	300	False
3/7/2025	20	False
3/6/2025	4000	True
3/5/2025	1500	False
3/4/2025	1400	False
3/3/2025	800	False
3/2/2025	200	False
3/1/2025	11400	True
2/28/2025	1600	False
2/27/2025	2200	False
2/26/2025	2400	False
2/25/2025	300	False
2/24/2025	900	False
2/23/2025	1400	False
3/14/2025	-218.6246337890625	0.5
3/13/2025	121.46533203125	1E-08
3/12/2025	637.060302734375	0.0006072842522107935
3/11/2025	-210.111572265625	0.23020433413886088
3/10/2025	588.001220703125	0.16687442411745346
3/9/2025	-673.347900390625	0.0466676693988578
3/8/2025	-1167.231689453125	0.03801085109784952
3/7/2025	-1320.1378173828125	0.13126578055792515
3/6/2025	2919.8409423828125	7.69350245974354E-05
3/5/2025	290.0047607421875	0.42574355705990735
3/4/2025	171.63330078125	0.43000990600955347
3/3/2025	-513.509521484375	0.4010561186557473
3/2/2025	-1231.407836914062